# Install Library and require

Create a .env file with a key called GEMINI_API_KEY to assign the value to your Gemini API Key.

In [1]:
!pip install ipywidgets==8.1.5
!pip install jupyter==1.1.1
!pip install google-generativeai==0.8.3
!pip install python-dotenv==1.0.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 15.9 MB/s eta 0:00:00
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached jsonschema_specifications-2024.10.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached referencing-0.35.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached cffi-1.17.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 27.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 25.0 MB/s eta 0:00:00a 0:00:01
Using cached httpcore-1.0.7-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 41.5 MB/s eta 0:00:0000:01
Using cached jsonschema-4.23.0-

# Find relevant laws of chunks in contract

## Get law database

In [2]:
import json

with open('Data/legal_corpus.json', 'r', encoding='utf-8') as f:
    law_data = json.load(f)
database = []
for laws in law_data:
    law_id = laws['law_id']
    for article in laws['articles']:
        article_id = article['article_id']
        title = article['title']
        text = article['text']
        database.append({"law_id": law_id, "article_id": article_id, "title": title, "text": text})
print(len(database))
print(database[0])

185
{'law_id': '個人情報の保護に関する法律（平成十五年法律第五十七号）/第一章\u3000総則', 'article_id': 1, 'title': '第一条\u3000目的', 'text': 'この法律は、デジタル社会の進展に伴い個人情報の利用が著しく拡大していることに鑑み、個人情報の適正な取扱いに関し、基本理念及び政府による基本方針の作成その他の個人情報の保護に関する施策の基本となる事項を定め、国及び地方公共団体の責務等を明らかにし、個人情報を取り扱う事業者及び行政機関等についてこれらの特性に応じて遵守すべき義務等を定めるとともに、個人情報保護委員会を設置することにより、行政機関等の事務及び事業の適正かつ円滑な運営を図り、並びに個人情報の適正かつ効果的な活用が新たな産業の創出並びに活力ある経済社会及び豊かな国民生活の実現に資するものであることその他の個人情報の有用性に配慮しつつ、個人の権利利益を保護することを目的とする。'}


## Get contract data

In [3]:
import json

with open('Data/chunked_contract.json', 'r', encoding='utf-8') as f:
    contract_data = json.load(f)

print(len(contract_data))
print(contract_data[0])

341
{'index': 6, 'text': '第１章\u3000総則\n（目的）\n第１条\u3000本個人情報取扱規程（以下「本規程」という。）は、会社が、個人情報の保護に関する法律（平成１５年法律第５７号）（以下「個人情報保護法」といい、個人情報保護法に基づき制定された政令、規則等を含むものとする。）及び行政手続における特定の個人を識別するための番号の利用等に関する法律（平成２５年法律第２７号）（以下「マイナンバー法」といい、マイナンバー法に基づき制定された政令、規則等を含むものとする。）に基づき、会社の取り扱う個人情報の適正な取扱いを確保するために定めるものである。', 'label': 'dieu', 'muc': '第１章\u3000総則', 'cap': '（目的）', 'dieu': '第１条\u3000本個人情報取扱規程（以下「本規程」という。）は、会社が、個人情報の保護に関する法律（平成１５年法律第５７号）（以下「個人情報保護法」といい、個人情報保護法に基づき制定された政令、規則等を含むものとする。）及び行政手続における特定の個人を識別するための番号の利用等に関する法律（平成２５年法律第２７号）（以下「マイナンバー法」といい、マイナンバー法に基づき制定された政令、規則等を含むものとする。）に基づき、会社の取り扱う個人情報の適正な取扱いを確保するために定めるものである。'}


## Config gemini

In [13]:
# Configure Gemini
import google.generativeai as genai
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

system_instruction = '''あなたは労働法を専門とする弁護士です。以下の文脈からユーザーの入力文の真偽を証明できるすべての法律を見つけてください。他の文脈外の法律や情報は使用しないでください。  
以下のJSONスキーマを使用してください：  
relevant_law = {  
  "law_id": str,  
  "article_id": int,  
  "title": str,  
  "text": str,  
  "explanation": str,  
  "label": bool  
}  
- "law_id"、"article_id"、"title"、"text"は文脈から取得してください。  
- "explanation"には、なぜその法律を選んだのかを日本語で記述してください。  
- "label"は、その法律がユーザーの文を真と証明する場合はtrue、偽と証明する場合はfalseを指定してください。  

適切な法律が見つからない場合は、空のリストを返してください。


文脈: %s
'''

api_key = os.getenv("GEMINI_API_KEY")

genai.configure(api_key=api_key)
safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE"
    }
]
model = genai.GenerativeModel('gemini-1.5-pro-latest',
                              safety_settings=safety_settings,
                              # generation_config={"response_mime_type": "application/json", "max_output_tokens": 8000},
                              system_instruction=system_instruction % (database))
chat = model.start_chat(history=[])

prompt_template = '''
ユーザー入力: 第３章 個人情報の取扱い 第１節　個人情報の取得・保有等
（利用目的の特定）
第２０条　会社は、業務を遂行するため必要な場合に限り、かつ目的の達成に必要な範囲に限って個人情報を取得するものとし、その利用の目的（以下「利用目的」という。）をできる限り特定するものとする。

回答: [
    {
        "law_id": "個人情報の保護に関する法律（平成十五年法律第五十七号）/第四章　個人情報取扱事業者等の義務等",
        "article_id": 17,
        "title": "第十七条　利用目的の特定",
        "text": "個人情報取扱事業者は、個人情報を取り扱うに当たっては、その利用の目的（以下「利用目的」という。）をできる限り特定しなければならない。個人情報取扱事業者は、利用目的を変更する場合には、変更前の利用目的と関連性を有すると合理的に認められる範囲を超えて行ってはならない。",
        "explanation": "この法律の第17条は、個人情報取扱事業者は、個人情報を取り扱う際には、その利用目的をできる限り特定しなければならないと定めています。これは、個人関連情報の取扱いにおいても同様であり、個人関連情報を取得、保有、利用する際には、その利用目的を明確に特定する必要があることを示唆しています。",
        "label": true
    }
]

ユーザー入力: %s

回答:
'''

list_result = []

## Generating

In [14]:
model = genai.GenerativeModel('gemini-1.5-pro-latest',
                              safety_settings=safety_settings)
chat = model.start_chat(history=[])
response = chat.send_message('hello', stream=True)
response.resolve()
print(response.text)
print('total token:', response.usage_metadata.total_token_count)

Hello there! How can I help you today?

total token: 13


#### one example

In [15]:
start_index = chunk_index = 16
# chunk_index += 1
print("chunk_index:", chunk_index)
contract = contract_data[chunk_index]
print(contract['text']+'\n')

chunk_index: 16
第１章　総則
（定義）
第４条　本規程において、次に掲げる用語の意義は、個人情報保護法の定義に従うものとする。
（１４）　個人関連情報データベース等



In [ ]:
# contract = {}

In [16]:
print("Prompt:")
print(prompt_template % (contract['text']))

Prompt:

ユーザー入力: 第３章 個人情報の取扱い 第１節　個人情報の取得・保有等
（利用目的の特定）
第２０条　会社は、業務を遂行するため必要な場合に限り、かつ目的の達成に必要な範囲に限って個人情報を取得するものとし、その利用の目的（以下「利用目的」という。）をできる限り特定するものとする。

回答: [
    {
        "law_id": "個人情報の保護に関する法律（平成十五年法律第五十七号）/第四章　個人情報取扱事業者等の義務等",
        "article_id": 17,
        "title": "第十七条　利用目的の特定",
        "text": "個人情報取扱事業者は、個人情報を取り扱うに当たっては、その利用の目的（以下「利用目的」という。）をできる限り特定しなければならない。個人情報取扱事業者は、利用目的を変更する場合には、変更前の利用目的と関連性を有すると合理的に認められる範囲を超えて行ってはならない。",
        "explanation": "この法律の第17条は、個人情報取扱事業者は、個人情報を取り扱う際には、その利用目的をできる限り特定しなければならないと定めています。これは、個人関連情報の取扱いにおいても同様であり、個人関連情報を取得、保有、利用する際には、その利用目的を明確に特定する必要があることを示唆しています。",
        "label": true
    }
]

ユーザー入力: 第１章　総則
（定義）
第４条　本規程において、次に掲げる用語の意義は、個人情報保護法の定義に従うものとする。
（１４）　個人関連情報データベース等

回答:



In [17]:
# start_index = chunk_index = 51
# chunk_index += 1
# contract = contract_data[chunk_index]
response = chat.send_message(prompt_template % (contract['text']), stream=True)
# response = model.generate_content(prompt_template % (contract['text']), stream=True)
# response = model.generate_content(system_instruction % (database[:len(database)//2]) + prompt_template % (contract['text']), stream=True)
# gemini_response  = ''.join([chunk.text for chunk in response]).strip().replace('```json', '').replace('```', '').replace("\n", '').replace(" ", '')
response.resolve()
print("response.text", response.text)
try:
    gemini_response = response.text.strip().replace('```json', '').replace('```', '')
    gemini_response = json.loads(gemini_response)
    list_result.append({"question": contract, "relevant_laws": gemini_response})
    print('total token:', response.usage_metadata.total_token_count)
    # print("chunk_index:", chunk_index)
    print(contract['text']+'\n')
    gemini_response
except: 
    print("No law found relating to the input contract sentence.")

response.text ```json
[
  {
    "law_id": "個人情報の保護に関する法律（平成十五年法律第五十七号）/第二章　定義",
    "article_id": 2,
    "title": "第二条　定義",
    "text": "この法律において、次の各号に掲げる用語の意義は、当該各号に定めるところによる。\n一　個人情報　生存する個人に関する情報であって、当該情報に含まれる氏名、生年月日その他の記述等により特定の個人を識別できるもの（他の情報と容易に照合することができ、それにより特定の個人を識別できることとなるものを含む。）をいう。\n二　個人情報データベース等　個人情報を含む情報の集合物であって、特定の個人情報を検索することができるように体系的に構成したもの（電磁的方法（電子的方法、磁気的方法その他の人の知覚によっては認識することができない方法をいう。次項第二号において同じ。）により構成されているものを含む。）をいう。\n三　個人情報取扱事業者　個人情報を取り扱う事業者をいう。\n四　第三者提供　個人情報取扱事業者が、その取り扱う個人情報を、本人以外の者に提供することをいう。\n五　オプトアウト　個人情報取扱事業者が、本人から、当該本人が識別される保有個人データの第三者への提供の停止を求められた場合であって、当該保有個人データの第三者への提供を停止すること（個人データの提供を受ける者の数が著しく少ないときその他第三者への提供を停止することが困難である場合として政令で定める場合を除く。）をいう。\n六　個人データ　特定の個人を識別することができる記述等（氏名、生年月日その他の記述等をいう。以下同じ。）により構成される記述等をいう。",
    "explanation": "この法律の第2条は、個人情報、個人情報データベース等、個人情報取扱事業者等の定義を定めています。社内規程でこれらの用語の定義が個人情報保護法に従うとされている場合、この条文が関連します。特に、個人関連情報データベース等は、この法律における個人情報データベース等の定義に準じて解釈されるべきです。",
    "label": true
  }
]
```

total token: 893
第１章　総則
（定義）
第４条　本規程において、次に掲げる用語

In [ ]:
# print("chunk_index:", chunk_index)
# print(contract['text']+'\n')
# print(response.text.strip().replace('```json', '').replace('```', ''))

In [18]:
with open('test.json', 'w', encoding='utf-8') as f:
    json.dump(list_result[-1], f, ensure_ascii=False, indent=4)

In [19]:
last_sent, last_received = chat.rewind()
print(len(chat.history))

2


copy thủ công phần response gán cho `relevevant_laws` dưới dạng `str`

In [21]:
relevant_laws = '''[
  {
    "law_id": "個人情報の保護に関する法律（平成十五年法律第五十七号）/第二章　定義",
    "article_id": 2,
    "title": "第二条　定義",
    "text": "この法律において、次の各号に掲げる用語の意義は、当該各号に定めるところによる。\n一　個人情報　生存する個人に関する情報であって、当該情報に含まれる氏名、生年月日その他の記述等により特定の個人を識別できるもの（他の情報と容易に照合することができ、それにより特定の個人を識別できることとなるものを含む。）をいう。\n二　個人情報データベース等　個人情報を含む情報の集合物であって、特定の個人情報を検索することができるように体系的に構成したもの（電磁的方法（電子的方法、磁気的方法その他の人の知覚によっては認識することができない方法をいう。次項第二号において同じ。）により構成されているものを含む。）をいう。\n三　個人情報取扱事業者　個人情報を取り扱う事業者をいう。\n四　第三者提供　個人情報取扱事業者が、その取り扱う個人情報を、本人以外の者に提供することをいう。\n五　オプトアウト　個人情報取扱事業者が、本人から、当該本人が識別される保有個人データの第三者への提供の停止を求められた場合であって、当該保有個人データの第三者への提供を停止すること（個人データの提供を受ける者の数が著しく少ないときその他第三者への提供を停止することが困難である場合として政令で定める場合を除く。）をいう。\n六　個人データ　特定の個人を識別することができる記述等（氏名、生年月日その他の記述等をいう。以下同じ。）により構成される記述等をいう。",
    "explanation": "この法律の第2条は、個人情報、個人情報データベース等、個人情報取扱事業者等の定義を定めています。社内規程でこれらの用語の定義が個人情報保護法に従うとされている場合、この条文が関連します。特に、個人関連情報データベース等は、この法律における個人情報データベース等の定義に準じて解釈されるべきです。",
    "label": true
  }
]'''
list_result.append({"question": contract, "relevant_laws": json.loads(relevant_laws.replace("\n", ''))})

#### full example

In [22]:
import time

start_index = 0
chunk_index = 0
for contract in contract_data[start_index:]:
    print("chunk_index:", start_index+chunk_index, '/', len(contract_data) - 1)
    print(contract['text']+'\n')
    # response = chat.send_message(prompt_template % (contract['text']), stream=True)
    response = model.generate_content(prompt_template % (contract['text']), stream=True)
    gemini_response  = ''.join([chunk.text for chunk in response]).strip().replace('```json', '').replace('```', '').replace("\n", '').replace(" ", '')
    gemini_response = json.loads(gemini_response)
    list_result.append({"question": contract, "relevant_laws": gemini_response})
    chunk_index += 1
    if chunk_index % 2 == 0:
        time.sleep(50)

chunk_index: 0 / 340
第１章　総則
（目的）
第１条　本個人情報取扱規程（以下「本規程」という。）は、会社が、個人情報の保護に関する法律（平成１５年法律第５７号）（以下「個人情報保護法」といい、個人情報保護法に基づき制定された政令、規則等を含むものとする。）及び行政手続における特定の個人を識別するための番号の利用等に関する法律（平成２５年法律第２７号）（以下「マイナンバー法」といい、マイナンバー法に基づき制定された政令、規則等を含むものとする。）に基づき、会社の取り扱う個人情報の適正な取扱いを確保するために定めるものである。

chunk_index: 1 / 340
第１章　総則
（解釈上の疑義）
第２条　本規程の解釈について疑義が生じた場合は、関係部署の長及び外部の専門家の意見を勘案し、会社がこれを決定する。



JSONDecodeError: Extra data: line 1 column 3 (char 2)

In [23]:
# if stream broke, delete the last conversation
last_sent, last_received = chat.rewind()
print(last_sent, last_received)

parts {
  text: "hello"
}
role: "user"
 parts {
  text: "Hello there! How can I help you today?\n"
}
role: "model"



In [24]:
print(response.text)

```json
[]
```

このユーザー入力は、社内規程の解釈に関するものであり、特定の個人情報保護法の条文に対応するものではありません。したがって、関連する法律の条文は見つからず、空のJSON配列が適切な回答となります。



In [25]:
print(len(list_result))

3


Nếu vẫn lỗi => save data rồi restart runtime, thay api key, sau đó chạy tiếp từ index gây lỗi

## Save data

In [29]:
import json

with open('./Data/find_law_gemini_contract.json', 'w', encoding='utf-8') as f:
    json.dump(list_result, f, ensure_ascii=False, indent=4)

In [33]:
import json

with open('./Data/find_law_gemini_contract.json', 'r') as f:
    list_result = json.load(f)
print(len(list_result))
print(list_result[0])

3
{'question': {'index': 18, 'text': '第１章\u3000総則\n（定義）\n第４条\u3000本規程において、次に掲げる用語の意義は、個人情報保護法の定義に従うものとする。\n（１４）\u3000個人関連情報データベース等', 'label': 'dieu', 'muc': '第１章\u3000総則', 'cap': '（定義）', 'dieu': '第４条\u3000本規程において、次に掲げる用語の意義は、個人情報保護法の定義に従うものとする。', 'con': '（１４）\u3000個人関連情報データベース等'}, 'relevant_laws': [{'law_id': '個人情報の保護に関する法律（平成十五年法律第五十七号）/第二章\u3000定義', 'article_id': 2, 'title': '第二条\u3000定義', 'text': 'この法律において、次の各号に掲げる用語の意義は、当該各号に定めるところによる。\n一\u3000個人情報\u3000生存する個人に関する情報であって、当該情報に含まれる氏名、生年月日その他の記述等により特定の個人を識別できるもの（他の情報と容易に照合することができ、それにより特定の個人を識別できることとなるものを含む。）をいう。\n二\u3000個人情報データベース等\u3000個人情報を含む情報の集合物であって、特定の個人情報を検索することができるように体系的に構成したもの（電磁的方法（電子的方法、磁気的方法その他の人の知覚によっては認識することができない方法をいう。次項第二号において同じ。）により構成されているものを含む。）をいう。\n三\u3000個人情報取扱事業者\u3000個人情報を取り扱う事業者をいう。\n四\u3000第三者提供\u3000個人情報取扱事業者が、その取り扱う個人情報を、本人以外の者に提供することをいう。\n五\u3000オプトアウト\u3000個人情報取扱事業者が、本人から、当該本人が識別される保有個人データの第三者への提供の停止を求められた場合であって、当該保有個人データの第三者への提供を停止すること（個人データの提供を受ける者の数が著しく少ないときその他第三者への提供を停止することが困難である場合として政令で定める場合を除く。）をいう。\n